# Redes Neurais
## Preâmbulo

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import time
import tensorflow as tf
import numpy as np
import numpy.random as nr
import pandas as pd

np.set_printoptions(precision=3, linewidth=100, suppress=True)

## Dataset

In [2]:
# -----------------------------------------------------------------------------------------------------------
# Wine Quality Data Set
# ---------------------
# [https://archive.ics.uci.edu/ml/datasets/Wine+Quality]
#
# 1. Title: Wine Quality 

# 2. Sources
#    Created by: Paulo Cortez (Univ. Minho), Antonio Cerdeira, Fernando Almeida, Telmo Matos and Jose Reis (CVRVV) @ 2009
   
# 3. Past Usage:

#   P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
#   Modeling wine preferences by data mining from physicochemical properties.
#   In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

#   In the above reference, two datasets were created, using red and white wine samples.
#   The inputs include objective tests (e.g. PH values) and the output is based on sensory data
#   (median of at least 3 evaluations made by wine experts). Each expert graded the wine quality 
#   between 0 (very bad) and 10 (very excellent). Several data mining methods were applied to model
#   these datasets under a regression approach. The support vector machine model achieved the
#   best results. Several metrics were computed: MAD, confusion matrix for a fixed error tolerance (T),
#   etc. Also, we plot the relative importances of the input variables (as measured by a sensitivity
#   analysis procedure).
 
# 4. Relevant Information:

#    The two datasets are related to red and white variants of the Portuguese "Vinho Verde" wine.
#    For more details, consult: http://www.vinhoverde.pt/en/ or the reference [Cortez et al., 2009].
#    Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables 
#    are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

#    These datasets can be viewed as classification or regression tasks.
#    The classes are ordered and not balanced (e.g. there are munch more normal wines than
#    excellent or poor ones). Outlier detection algorithms could be used to detect the few excellent
#    or poor wines. Also, we are not sure if all input variables are relevant. So
#    it could be interesting to test feature selection methods. 

# 5. Number of Instances: red wine - 1599; white wine - 4898. 

# 6. Number of Attributes: 11 + output attribute
  
#    Note: several of the attributes may be correlated, thus it makes sense to apply some sort of
#    feature selection.

# 7. Attribute information:

#    For more information, read [Cortez et al., 2009].

#    Input variables (based on physicochemical tests):
#    1 - fixed acidity
#    2 - volatile acidity
#    3 - citric acid
#    4 - residual sugar
#    5 - chlorides
#    6 - free sulfur dioxide
#    7 - total sulfur dioxide
#    8 - density
#    9 - pH
#    10 - sulphates
#    11 - alcohol
#    Output variable (based on sensory data): 
#    12 - quality (score between 0 and 10)

# 8. Missing Attribute Values: None
# -----------------------------------------------------------------------------------------------------------
import os
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

wine_data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-{}.csv"
wine_data_file = '../data/winequality-{}.csv'

for red_or_white in ['red', 'white']:
    if not os.path.isfile(wine_data_file.format(red_or_white)):
        # Read wine quality data from UCI website
        req = requests.get(wine_data_url.format(red_or_white))
        open(wine_data_file.format(red_or_white), 'w').write(req.text)

df_white = pd.read_csv("../data/winequality-white.csv", sep=';')
dfw = df_white.as_matrix()

scaler = StandardScaler()
X = scaler.fit_transform(dfw[:,:-1])
y = dfw[:,-1]

m, n = X.shape

X = np.hstack((np.ones((m, 1)), X))

Xtra, Xval, ytra, yval = train_test_split(X, y.reshape(-1, 1), train_size=0.8, random_state=20170421)

print('Shapes:', Xtra.shape, Xval.shape, ytra.shape, yval.shape)
print('Target:', y.min(), y.max(), y.mean())

Shapes: (3918, 12) (980, 12) (3918, 1) (980, 1)
Target: 3.0 9.0 5.87790935076


In [3]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import fetch_california_housing
# housing = fetch_california_housing()

# print('Description')
# print('-----------')
# print(housing.DESCR)
# print('Features')
# print('--------')
# print(housing.feature_names)
# print()
# print('Shapes')
# print('------')
# print(housing.data.shape, housing.target.shape)
# print()

# m, n = housing.data.shape

# scaler = StandardScaler()
# scaled_housing_data = scaler.fit_transform(housing.data)

# housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# X = housing_data_plus_bias / housing_data_plus_bias.max(0)
# y = housing.target

# Xtra, Xval, ytra, yval = train_test_split(X, y.reshape(-1, 1), train_size=0.8, random_state=20170412)

# print('Shapes:', Xtra.shape, Xval.shape, ytra.shape, yval.shape)
# print('Target:', y.min(), y.max(), y.mean())

## Implementação matricial
### O código

In [65]:
import numpy as np
from scipy.optimize import fmin_bfgs, fmin_ncg, fmin_tnc
from scipy.optimize.tnc import RCSTRINGS

class BackPropNeuralNetwork:
    
    def __init__(self, layer_sizes=[], is_classifier=True, lamb=0.0):
        self.L = len(layer_sizes)
        self.s = layer_sizes
        self.Theta = None
        self.classifier = is_classifier
        self.set_lambda(lamb)

    def save(self, filename):
        import pickle as pickle
        data = {
            'L':      self.L,
            's':      self.s,
            'Theta' : self.Theta,
            'lambda': self.lambda_,
        }
        pickle.dump(data, open(filename, 'w'))
    
    def load(self, filename):
        import pickle as pickle
        data = pickle.load(open(filename, 'r'))
        self.L = data['L']
        self.s = data['s']
        self.Theta = data['Theta']
        self.lambda_ = data['lambda']
        
    def init_theta(self, epsilon=None):
        from numpy.random import rand
        self.Theta = []
        for i in range(self.L-1):
            if epsilon is None:
                eps = np.sqrt(6.0 / (self.s[i] + self.s[i+1]))
            else:
                eps = epsilon
            self.Theta.append(2*eps*rand(self.s[i+1], self.s[i]+1) - eps)
        return self._unroll(self.Theta)

    def set_lambda(self, value=None):
        if value is not None:
            self.lambda_ = value
        return self.lambda_
        
    def tr_newton(self, X, y, nfeval=200, pgtol=-1, xtol=-1, ftol=-1, disp=1):
        thetas = self._unroll(self.Theta)
        thetas, nfeval, rc = fmin_tnc(self.compute_cost_and_gradient,
                                      thetas, fprime=None, args=(X, y),
                                      disp=disp, maxfun=nfeval, xtol=xtol,
                                      ftol=ftol, pgtol=pgtol)
        self.Theta = self._roll(thetas)
        return nfeval, RCSTRINGS[rc]

    def sgd(self, X, y, lr, momentum, batch, nepochs):
        r = nepochs / 10
        m, n = X.shape
        n_batches = int(np.ceil(m / batch))
        
        Velocity = [np.zeros_like(th) for th in self.Theta]
        for epoch in range(nepochs):
            for ii in range(n_batches):
                kk = batch * ii
                X_batch, y_batch = X[kk:kk+batch], y[kk:kk+batch]
                
                cost, Delta = self._compute_cost_and_gradient(self.Theta, X, y)
                for i in range(len(self.Theta)):
                    V = momentum * Velocity[i] - lr * Delta[i]
                    Velocity[i] = V
                    self.Theta[i] += V
                    
            if not epoch % r:
                print(('{:4d} Cost: {:.5f}'.format(epoch, cost)))
        print(('{:4d} Cost: {:.5f}'.format(epoch, cost)))
    
    def predict(self, X):
        a = self._compute_activations(self.Theta, X)
        return a[-1]
    
    def compute_cost_and_gradient(self, thetas, X, y):
        Theta = self._roll(thetas)
        J, Delta = self._compute_cost_and_gradient(Theta, X, y)
        return J, self._unroll(Delta)

    def _compute_cost_and_gradient(self, Theta, X, y):
        M, N  = X.shape
        Delta = [None for n in self.s[:-1]]
        # Forward propagation
        a = self._compute_activations(Theta, X)
        # Back propagation
        d = self._compute_errors(Theta, a, y)
        # Cost computation
        if self.classifier:
            # classifier: binary cross-entropy
            J = - (y * np.log(a[-1]) + (1 - y) * np.log(1 - a[-1])).sum() / M
        else:
            # regressor: mean squared error
            J = 0.5 * np.square(y - a[-1]).sum() / M
        for j in range(self.L-1):
            # ... add regularization to cost
            J += 0.5 * self.lambda_ * (Theta[j][:,1:] * Theta[j][:,1:]).sum() / M
            # gradient computation
            Delta[j] = np.dot(np.transpose(d[j+1]), a[j]) / M
            # ... add regularization to gradient
            Delta[j][:,1:] += self.lambda_ * Theta[j][:,1:] / M
        return J, Delta

    def _compute_activations(self, Theta, X):
        a = [None for n in self.s]
        a[0] = X
        for j in range(1, self.L):
            a[j-1] = np.insert(a[j-1], 0, 1, 1)
            z = np.dot(a[j-1], np.transpose(Theta[j-1]))
            if j == self.L-1 and not self.classifier:
                a[j] = z
            else:
                a[j] = logistic(z)
        return a            

    def _compute_errors(self, Theta, a, y):
        d = [None for n in self.s]
        d[-1] = a[-1] - y
        for j in range(self.L-2, 0, -1):
            d[j] = np.dot(d[j+1], Theta[j]) * a[j] * (1 - a[j])
            d[j] = d[j][:,1:]
        return d
        
    def _unroll(self, Theta):
        thetas = np.concatenate([t.flat for t in Theta])
        return thetas
    
    def _roll(self, thetas):
        Theta, m = [], 0
        for l in range(self.L-1):
            h, w = self.s[l+1], self.s[l] + 1
            n = w * h
            Theta.append(thetas[m:m+n].reshape((h,w)))
            m += n
        return Theta

    def compute_approx_gradient(self, thetas, X, y, eps=0.0001):
        agrads = np.zeros_like(thetas)    
        for i in range(thetas.shape[0]):
            tplus = thetas.copy()
            tplus[i] += eps
            tminus = thetas.copy()
            tminus[i] -= eps
            Jplus,  G = self.compute_cost_and_gradient(tplus, X, y)
            Jminus, G = self.compute_cost_and_gradient(tminus, X, y)
            agrads[i] = (Jplus - Jminus) / (2*eps)
        return agrads
    
    def learning_curve(self, Xtrain, ytrain, Xval, yval, lambd, incr=10, nfeval=100, prnt=1):
        m, n = Xtrain.shape
        J = np.zeros((m/incr,3), np.float64)
        for i, n in enumerate(range(incr, m, incr)):
            self.init_theta()
            self.set_lambda(lambd)
            self.train(Xtrain[:n,:], ytrain[:n,:], nfeval=nfeval, disp=0)
            J[i,0], G = self._compute_cost_and_gradient(self.Theta, Xtrain[:n,:], ytrain[:n,:])
            self.set_lambda(0)
            J[i,1], G = self._compute_cost_and_gradient(self.Theta, Xtrain[:n,:], ytrain[:n,:])
            J[i,2], G = self._compute_cost_and_gradient(self.Theta, Xval, yval)
            if prnt: print((i, n, J[i]))
        return J

def logistic(z):
    z = np.asarray(z)
    z = np.minimum(z,  15)
    z = np.maximum(z, -15)
    return np.ones(z.shape)/(1.0 + np.exp(-z))

def gradient_check(lamb, classif):
    def norm(a):
        return np.sqrt(np.sum(a*a))
    nn = BackPropNeuralNetwork([4, 5, 3], lamb=lamb, is_classifier=classif)
    thetas = nn.init_theta()
    X = np.sin(np.arange(32)).reshape((8, 4)) / 10
    y = np.zeros((8, 3))
    for i, k in enumerate(np.mod(np.arange(8), 3)):
        y[i,k] = 1.0
    agrad = nn.compute_approx_gradient(thetas, X, y)
    J, grad = nn.compute_cost_and_gradient(thetas, X, y)
    diff = norm(agrad-grad) / norm(agrad+grad)
    return diff

print(gradient_check(0, True))
print(gradient_check(0, False))

1.57690004466e-10
6.20300197406e-11


In [10]:
layer_sizes = [n+1, 40, 20, 1]
nepochs = 1000

### Teste com otimizador avançado (*Truncated Newton*)

In [11]:
nnet = BackPropNeuralNetwork(layer_sizes, is_classifier=False)
nnet.init_theta();

try:
    t0 = time.time()
    nnet.tr_newton(Xtra, ytra, nepochs)
    t1 = time.time()
except KeyboardInterrupt:
    pass

print('\nTrained in {:2f}s'.format(t1-t0))
yhat = nnet.predict(Xval)
mse = np.square(yhat - yval).mean()
print('\nMSE:', mse)
print()
print(yhat[:10,0])
print(yval[:10,0])


Trained in 17.602465s

MSE: 0.739397318578

[ 5.687  6.286  6.781  5.804  5.628  7.941  6.423  6.08   7.18   6.145]
[ 6.  6.  7.  5.  6.  8.  7.  5.  8.  6.]


### Teste com gradiente descendente

In [12]:
nnet = BackPropNeuralNetwork(layer_sizes, is_classifier=False)
nnet.init_theta();

try:
    t0 = time.time()
    nnet.sgd(Xtra, ytra, 0.2, 0.9, 4128, nepochs)
    t1 = time.time()
except KeyboardInterrupt:
    pass

print('\nTrained in {:2f}s'.format(t1-t0))
yhat = nnet.predict(Xval)
mse = np.square(yhat - yval).mean()
print('\nMSE:', mse)
print()
print(yhat[:10,0])
print(yval[:10,0])

   0 Cost: 16.10587
 100 Cost: 0.28633
 200 Cost: 0.27427
 300 Cost: 0.25655
 400 Cost: 0.24620
 500 Cost: 0.23956
 600 Cost: 0.23460
 700 Cost: 0.23040
 800 Cost: 0.22677
 900 Cost: 0.22361
 999 Cost: 0.22084

Trained in 17.153527s

MSE: 0.480271455107

[ 5.628  6.85   6.771  5.363  5.773  6.739  6.748  4.918  6.555  6.607]
[ 6.  6.  7.  5.  6.  8.  7.  5.  8.  6.]


## Keras

In [58]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

estop = EarlyStopping(patience=50, verbose=1)

layer_sizes = [n+1, 40, 20, 1]
nepochs = 1000

def build():
    model = Sequential()
    model.add(Dense(40, activation='sigmoid', input_shape=(n,)))
    model.add(Dense(20, activation='sigmoid'))
    model.add(Dense(1, activation=None))
    return model

nnet = build()
nnet.summary()

opt = SGD(lr=0.1, momentum=0.9)
nnet.compile(loss="mse", optimizer=opt)    

Xt = Xtra[:, 1:]
Xv = Xval[:, 1:]

try:
    t0 = time.time()
    histo2 = nnet.fit(Xtra[:,1:], ytra, batch_size=516, epochs=nepochs, verbose=0, 
                      validation_data=(Xval[:,1:], yval), callbacks=[estop])
    t1 = time.time()
except KeyboardInterrupt:
    pass

print('\nTrained in {:2f}s'.format(t1-t0))
print()
yhat = nnet.predict(Xv, verbose=0)
mse = np.square(yhat - yval).mean()
print('MSE:', mse)
print()
print(yhat[:10,0])
print(yval[:10,0])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_79 (Dense)             (None, 40)                480       
_________________________________________________________________
dense_80 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_81 (Dense)             (None, 1)                 21        
Total params: 1,321
Trainable params: 1,321
Non-trainable params: 0
_________________________________________________________________
Epoch 00189: early stopping

Trained in 2.556202s

MSE: 0.485530365783

[ 5.475  6.827  6.693  5.29   5.684  6.715  6.773  5.09   6.454  6.409]
[ 6.  6.  7.  5.  6.  8.  7.  5.  8.  6.]


In [59]:
print()
acc = 100.0 * np.where(np.abs(yhat - yval) < 0.5, 1, 0).sum() / yval.shape[0]
print('Accuracy: {:.2f}%'.format(acc))


Accuracy: 54.80%


## Equações
### Backpropagation
----

Para a última camada, temos:

$$\frac{\partial J}{\partial w_{ij}^{(L)}} = \frac{\partial J}{\partial a_i^{(L)}} \cdot \frac{\partial a_i^{(L)}}{\partial z_i^{(L)}} \cdot \frac{\partial z_i^{(L)}}{\partial w_{ij}^{(L)}}$$

$$\frac{\partial z_i^{(L)}}{\partial w_{ij}^{(L)}} = a_j^{(L-1)}$$

$$\delta_i^{(L)} = \frac{\partial J}{\partial a_i^{(L)}} \cdot \frac{\partial a_i^{(L)}}{\partial z_i^{(L)}}$$

$$\frac{\partial J}{\partial w_{ij}^{(L)}} = \delta_i^{(L)} \cdot a_j^{(L-1)}$$

Passando para a próxima camada:

$$\frac{\partial J}{\partial w_{ij}^{(L-1)}} = \frac{\partial J}{\partial a_i^{(L)}} \cdot \frac{\partial a_i^{(L)}}{\partial z_i^{(L)}} \cdot \frac{\partial z_i^{(L)}}{\partial a_i^{(L-1)}}  \cdot \frac{\partial a_i^{(L-1)}}{\partial z_i^{(L-1)}} \cdot \frac{\partial z_i^{(L-1)}}{\partial w_{ij}^{(L-1)}}$$

$$\frac{\partial J}{\partial w_{ij}^{(L-1)}} = \delta_i^{(L)} \cdot \frac{\partial z_i^{(L)}}{\partial a_i^{(L-1)}} \cdot \frac{\partial a_i^{(L-1)}}{\partial z_i^{(L-1)}} \cdot \frac{\partial z_i^{(L-1)}}{\partial w_{ij}^{(L-1)}}$$


------

Para a regressão linear, temos:

$$J = \frac{1}{2} \sum_{k}^{} (\hat{y}_k - y_k)^2  \Rightarrow   \frac{\partial J}{\partial a_i} = a_i - y_i$$ 
$$a_i = z_i \Rightarrow \frac{\partial a_i}{\partial z_i} = 1$$

Para a regressão logística:

$$J = -  \sum_{k}^{} (y_k \log{\hat{y}_k} + (1 - y_k) \log{(1 - \hat{y}_k)}) \Rightarrow \frac{\partial J}{\partial a_i} = \frac{a_i - y_i}{a_i (1 - a_i)}$$ 
$$a_i = \frac{1}{1 + e^{-z_i}} \Rightarrow \frac{\partial a_i}{\partial z_i} = a_i (1 - a_i)$$

Enfim:

$$\delta = a_i - y_i  \Rightarrow  \frac{\partial J}{\partial w_{ij}} = \delta x_j$$


### Propagação para trás

---- 
\begin{align*} 
\frac{\partial J}{\partial W^{(L)}} & = \frac{\partial J}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial W^{(L)}} \\
\\
\frac{\partial z^{(L)}}{\partial W^{(L)}} & = (a^{(L-1)})^T \\
\\
\text{Definindo   } \delta^{(L)} & = \frac{\partial J}{\partial z^{(L)}} \\
 & = \frac{\partial J}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \\
 & = \frac{a^{(L)} - y}{ (1 - a^{(L)}) \cdot a^{(L)}} \cdot (1 - a^{(L)}) \cdot a^{(L)} \\
\\
\delta^{(L)} & = a^{(L)} - y \\
\\
\frac{\partial J}{\partial W^{(L)}} & = \delta^{(L)} \cdot (a^{(L-1)})^T
\end{align*}
-----

Passando para a próxima camada:
-----
\begin{align*} 
\frac{\partial J}{\partial W^{(L-1)}} & = \frac{\partial J}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial a^{(L-1)}}  \cdot \frac{\partial a^{(L-1)}}{\partial z^{(L-1)}} \cdot \frac{\partial z^{(L-1)}}{\partial W^{(L-1)}} \\
 & = (W^{(L)})^T \cdot \delta^{(L)} \circ  (1 - a^{(L-1)}) a^{(L-1)} \cdot (a^{(L-2)})^T \\
\\
\delta^{(L-1)} & = (W^{(L)})^T \cdot \delta^{(L)} \circ (1 - a^{(L-1)}) a^{(L-1)} \\
\\
\frac{\partial J}{\partial W^{(L-1)}} & = \delta^{(L-1)} \cdot a^{(L-2)}
\end{align*}
-----


### Propagação para trás (Backpropagation)

Uma descrição mais detalhada do processo de *backpropagation* pode ser encontrada em 
<a href="https://medium.com/becoming-human/back-propagation-is-very-simple-who-made-it-complicated-97b794c97e5c">neste artigo</a>

Para a última camada, temos:

\begin{align*} 
\frac{\partial J}{\partial w_{ij}^{(L)}} &= \frac{\partial J}{\partial a_i^{(L)}} \cdot \frac{\partial a_i^{(L)}}{\partial z_i^{(L)}} \cdot \frac{\partial z_i^{(L)}}{\partial w_{ij}^{(L)}} \\
\\
\frac{\partial z_i^{(L)}}{\partial w_{ij}^{(L)}} &= a_j^{(L-1)} \\
\\
\delta_i^{(L)} &= \frac{\partial J}{\partial a_i^{(L)}} \cdot \frac{\partial a_i^{(L)}}{\partial z_i^{(L)}} = \frac{a_i^{(L)} - y_i}{a_i^{(L)} (1 - a_i^{(L)})} \cdot a_i^{(L)} (1 - a_i^{(L)}) \\
\\
\delta_i^{(L)} &= a_i^{(L)} - y_i \\
\\
\frac{\partial J}{\partial w_{ij}^{(L)}} &= \delta_i^{(L)} \cdot a_j^{(L-1)}
\end{align*}

Passando para a próxima camada:

\begin{align*} 
\frac{\partial J}{\partial w_{ij}^{(L-1)}} &= \frac{\partial J}{\partial a_i^{(L-1)}} \cdot \frac{\partial a_i^{(L-1)}}{\partial z_i^{(L-1)}}  \cdot  \frac{\partial z_i^{(L-1)}}{\partial w_{ij}^{(L-1)}}
\\
\frac{\partial z_i^{(L-1)}}{\partial w_{ij}^{(L-1)}} &= a_j^{(L-2)}
\\
\frac{\partial a_i^{(L-1)}}{\partial z_i^{(L-1)}} &= a_i^{(L-1)} (1 - a_i^{(L-1)})
\\
\frac{\partial J}{\partial a_{i}^{(L-1)}} &= \sum_{k = 0}^{n_L} \frac{\partial J}{\partial z_{k}^{(L)}} \cdot \frac{\partial z_k^{(L)}}{\partial a_{i}^{(L-1)}} = \sum_{k = 0}^{n_L} \delta_k^{L} \cdot w_{ki}^{L}
\\
\frac{\partial J}{\partial w_{ij}^{(L-1)}} &= \sum_{k = 0}^{n_L} \delta_k^{L} \cdot w_{ik}^{L} \cdot a_i^{(L-1)} (1 - a_i^{(L-1)}) \cdot a_j^{(L-2)} \\
\\
\end{align*}